# Description

This code defines a Pydantic model and provides methods to convert the model to YAML format,
including field descriptions as comments, and to load the model from a YAML file.

In [1]:
from pydantic import BaseModel, Field
import json
import yaml

class MyModel(BaseModel):
    name: str = Field(description="The name of the item")
    count: int = Field(description="Number of items")
    active: bool = Field(description="Whether the item is active")
    
    def get_descriptions(self) -> dict:
        schema = self.model_json_schema()
        return {field: field_info.get('description', '') for field, field_info in schema["properties"].items()}
    
    def to_yaml(self):
        # Get the model's JSON schema
        return json.loads(self.model_dump_json())
    
    def to_yaml_file(self, file_path):
        # Convert to YAML
        output = self.to_yaml()
        descriptions = self.get_descriptions()
        
        # Write to file with descriptions as a block comment at the top
        with open(file_path, 'w') as file: 
            for key, value in output.items():
                if key in descriptions:
                    file.write(f"# {key}: {descriptions[key]}\n")
            
            # then write out full object
            yaml.dump(output, file, sort_keys=False)
    
    @classmethod
    def from_yaml(cls, yaml_str):
        data = yaml.safe_load(yaml_str)
        return cls(**data)
    
    @classmethod
    def from_yaml_file(cls, file_path):
        with open(file_path, 'r') as file:
            return cls.from_yaml(file.read())

In [2]:
# Example usage
item = MyModel(name="test", count=5, active=True)
print(item)

name='test' count=5 active=True


In [3]:
print(item.to_yaml())
item.to_yaml_file('output.yaml')

{'name': 'test', 'count': 5, 'active': True}


In [4]:
loaded = MyModel.from_yaml_file('output.yaml')

In [5]:
loaded == item  # Should be True if loaded correctly

True

# show what file looks like

In [6]:
with open('output.yaml', 'r') as file:
    print(file.read())

# name: The name of the item
# count: Number of items
# active: Whether the item is active
name: test
count: 5
active: true


# load with normal yaml

In [7]:
with open('output.yaml', 'r') as file:
    loaded = yaml.safe_load(file)

In [8]:
loaded

{'name': 'test', 'count': 5, 'active': True}